In [ ]:
import matplotlib as mpl
# COLOR = 'white'
# mpl.rcParams['text.color'] = COLOR
# mpl.rcParams['axes.labelcolor'] = COLOR
# mpl.rcParams['xtick.color'] = COLOR
# mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['figure.dpi'] = 100


# import fancyimpute as fi
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import variation
from math import isnan

from scipy.cluster.hierarchy import dendrogram,linkage

import sys
sys.path.append("../src/")
import lumberjack
import tree_reader as tr

import sklearn

In [ ]:
counts = np.loadtxt('/home/bbrener1/lumberprot/work/nesterowa_counts.txt')
header = np.loadtxt('/home/bbrener1/lumberprot/work/nesterowa_gene_header.txt',dtype=str)
annotations = 

plt.figure(figsize=(15,10))
plt.title("Cell x Gene Expression Unsorted",fontsize=20)
plt.imshow(counts,aspect='auto')
plt.xlabel("Genes",fontsize=15)
plt.ylabel("Cells",fontsize=15)
plt.colorbar()
plt.show()

cell_sort = dendrogram(linkage(counts,metric='cos',method='average'),no_plot=True)['leaves']
feature_sort = dendrogram(linkage(counts.T,metric='cos',method='average'),no_plot=True)['leaves']

plt.figure(figsize=(15,10))
plt.title("Cell x Gene Expression, Agglomerative",fontsize=20)
plt.imshow(counts[cell_sort].T[feature_sort].T,aspect='auto')
plt.xlabel("Genes",fontsize=15)
plt.ylabel("Cells",fontsize=15)
plt.colorbar()
plt.show()

plt.figure()
plt.title("Frequency of Mean Gene Expression Values, Mouse Blood Cells")
plt.xlabel("Mean Expression (log TPM)")
plt.ylabel("Frequency")
plt.hist(np.mean(counts,axis=0),bins=50,log=True)
plt.show()

plt.figure()
plt.title("Frequency of Individual Expression Values, Mouse Blood Cells")
plt.xlabel("Mean Expression (log TPM)")
plt.ylabel("Frequency")
plt.hist(counts.flatten(),bins=50,log=True)
plt.show()


# print(np.mean(counts,axis=0).shape)
# print(header.shape)

## GOAL: EVALUATE CONSISTENCY OF THE FORESTS GENERATED, AND CLUSTERING ON THEIR BASIS

In [ ]:
!ls
!ls ./consistency_analysis/
# !mkdir ./consistency_analysis

In [ ]:
## We want to train ~10 forests on the same data and examine their consistency. 

forests = []

for i in range(10): 

    forest = lumberjack.fit(counts,trees=100,dispersion_mode="ssme",norm="l1",lrg_mem=True,drop='none',sfr=1,in_feature_subsample=1000,out_feature_subsample=1000,sample_subsample=200,depth=5,leaves=20,header=header)
    forest.backup(f'./consistency_analysis/forest_{i}')
    
    forests.append(forest)



In [ ]:
# We must now compare the 10 different forests to examine different attributes for consistency. (Is a forest with 100 trees going to be good at consistent predictions?)

# We can test error of the prediction for each individual sample
# We can also test the consistency of the clustering performed by the forest
# We can also see if forests rediscover the same structural properties

forests = []

# for i in range(10): 
#     forests.append(tr.Forest.reconstitute(f'./consistency_analysis/forest_{i}'))

for i in range(10): 
    forests.append(tr.Forest.reconstitute(f'./consistency_analysis/forest_{i}_cached'))


In [ ]:
!ls consistency_analysis/
# !mkdir consistency_analysis/html_summaries
# !rm -r consistency_analysis/html_summaries/*

In [ ]:
for i,forest in enumerate(forests):
    
    forest.set_cache(True)
    forest.interpret_splits(depth=4,pca=False,sub=.5,metric='cos',mode='additive')
    forest.most_likely_tree()
    forest.html_tree_summary()
    !rm -r ./consistency_analysis/html_summaries/forest_{i}
    !mkdir ./consistency_analysis/html_summaries/forest_{i}
    !cp -r ../html/* ./consistency_analysis/html_summaries/forest_{i}

In [ ]:
for i,forest in enumerate(forests):
    forest.reset_sample_clusters()
    forest.cluster_samples_encoding(sub=.5,pca=False,override=True)    
    fig = forest.plot_cell_clusters()
    fig.savefig(f'./consistency_analysis/forest_{i}.png')

In [ ]:
# for i,forest in enumerate(forests):
#     forest.backup(f'./consistency_analysis/forest_{i}_cached')

In [ ]:
from sklearn.preprocessing import OneHotEncoder

f1 = OneHotEncoder().fit_transform(forests[0].sample_labels.reshape(-1, 1))
f2 = OneHotEncoder().fit_transform(forests[1].sample_labels.reshape(-1, 1))

In [ ]:
from scipy.spatial.distance import squareform,pdist,cdist

distances = cdist(np.array(f1.todense()).T,np.array(f2.todense()).T,metric="jaccard")
distances.shape

In [ ]:
a1_sort = dendrogram(linkage(distances,metric='correlation',method='average'),no_plot=True)['leaves']
a2_sort = dendrogram(linkage(distances.T,metric='correlation',method='average'),no_plot=True)['leaves']


# plt.figure()
# plt.imshow(distances[a1_sort].T[a2_sort],cmap='binary')
# plt.colorbar()
# plt.show()

plt.figure()
plt.title("Inter-run Cluster Similarities (Jaccard Index)")
plt.xlabel("Clusters")
plt.ylabel("Clusters")
plt.imshow(distances,cmap='binary')
plt.colorbar()
plt.show()

In [ ]:
# from sklearn.metrics import normalized_mutual_info_score

# pairwise = np.zeros((11,11))

# for i in range(10):
#     for j in range(i,10):
#         score = normalized_mutual_info_score(forests[i].sample_labels,forests[j].sample_labels)
#         pairwise[i,j] = score
#         pairwise[j,i] = score

# shuffled = forests[0].sample_labels.copy()
# np.random.shuffle(shuffled)

# for i in range(10):
#     score = normalized_mutual_info_score(forests[i].sample_labels,shuffled)
#     pairwise[i,10] = score
#     pairwise[10,i] = score
    
plt.figure()
plt.title("Cluster Partition Mutual Information")
plt.imshow(pairwise)
plt.colorbar()
plt.show()


In [ ]:
normalized_mutual_info_score(forests[0].sample_labels,forests[1].sample_labels)

In [ ]:
forests[0].reset_sample_clusters()
forests[0].cluster_samples_encoding()

In [ ]:
## Let's evaluate a more difficult dataset: CITESeq

In [ ]:
counts = np.loadtxt('/home/bbrener1/lumberprot/work/citeseq_cbmc_counts_1000.tsv')
header = np.loadtxt('/home/bbrener1/lumberprot/work/citeseq_cbmc_header_1000.txt',dtype=str)


In [ ]:
# We would like to evaluate basic inference parameters on a difficult target. 
# To this end we will build 12 different forests of 100 trees each and evaluate their performance in clustering the dataset samples

# Parameteres we will vary initially:
# Dispersion: ssme vs var
# Norm between features: L1 vs L2
# SFR: 1.0,0.5,0.

# After we evaluate these using defaults for other values, we will examine sampling rate

In [ ]:
forests = []
for dispersion in ["ssme","var"]:
    for norm in ["l1","l2"]:
        for sfr in [1.0,0.5,0.]:
            forest = lumberjack.fit(counts,trees=100,dispersion_mode=dispersion,norm=norm,lrg_mem=True,drop='none',sfr=sfr,in_feature_subsample=300,out_feature_subsample=300,sample_subsample=100,depth=5,leaves=20,header=header)
            forest.backup(f'./consistency_analysis/forest_parameters/forest_{dispersion}_{norm}_{sfr}')
            forests.append(forest)


In [ ]:
for dispersion in ["ssme","var"]:
    for norm in ["l1","l2"]:
        for sfr in [1.0,0.5,0.]:
            forest = tr.Forest.reconstitute(f'./consistency_analysis/forest_parameters/forest_{dispersion}_{norm}_{sfr}')
            forest.cluster_samples_encoding(sub=.8,k=10,metric="jaccard",no_plot=True)
            forest.plot_sample_clusters()

In [ ]:
!mkdir ./consistency_analysis/forest_parameters

In [ ]:
!ls ./consistency_analysis/forest_parameters/

In [ ]:
f_s_l2_05 = tr.Forest.reconstitute("consistency_analysis/forest_parameters/forest_ssme_l2_0.5")

In [ ]:
f_s_l2_05.reset_sample_clusters()
f_s_l2_05.cluster_samples_encoding(sub=.5,k=20,metric="jaccard")
f_s_l2_05.plot_sample_clusters()

In [ ]:
forest = lumberjack.fit(counts,trees=100,dispersion_mode='ssme',norm='l2',lrg_mem=True,drop='none',sfr=.5,in_feature_subsample=300,out_feature_subsample=300,sample_subsample=100,depth=10,leaves=20,header=header)


In [ ]:
forest.set_cache(True)

In [ ]:
forest.reset_sample_clusters()
forest.cluster_samples_encoding(sub=.5,k=20,metric="jaccard")

In [ ]:
forest.plot_sample_clusters()

In [ ]:
forest.output.shape

In [ ]:
forest.backup("nesterowa_l2_small_forest")